In [43]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_absolute_error
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
from pathlib import Path
import os
from datetime import datetime
import pickle

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:
device

device(type='cuda')

In [46]:
work_dir =  os.path.join(os.getcwd(), "historical_projections")
Path(work_dir).mkdir(parents=True, exist_ok=True)

## Load data and prepare for inference / eval

In [4]:
data_file = os.path.join(os.getcwd(), "CAISO_Data_2019_2021_NN.with_historical_projections.csv")
# read in the processed data
data_df = pd.read_csv(data_file, index_col=0)
data_df.index = pd.to_datetime(data_df.index)
# add temporal features
## Temporal features
data_df.loc[:,"Day_of_Year"] = [instant.timetuple().tm_yday for instant in data_df.index]
data_df.loc[:,"Hour"] = data_df.index.hour
# number of hours since earliest data point
def hours_since_2018(instant):
    return (instant - data_df.index[0]).total_seconds() // 3600
data_df.loc[:,"Hours_since_2018"] = [hours_since_2018(instant) for instant in data_df.index]


# load the scaler to use for scaling the data prior to inference
scaler_file = os.path.join(os.getcwd(), "FF_models", "scaler.hours_since_2018.pkl")
with open(scaler_file, "rb") as f:
    scaler = pickle.load(f)

Not all hours have a projected value for day-ahead or hour-ahead demand/vre. Only evaluate on the hours that have projected values for both demand and vre.

In [28]:
data_sets = {"hour_ahead": {"demand_col": "Hour_Ahead_Forecast_Demand",
                            "vre_col": "Hour_Ahead_Forecast_VRE",
                            "d_demand_col": "Delta_Hour_Ahead_Forecast_Demand",
                            "d_vre_col": "Delta_Hour_Ahead_Forecast_VRE",
                           }, 
             "day_ahead": {"demand_col": "Day_Ahead_Forecast_Demand",
                           "vre_col": "Day_Ahead_Forecast_VRE",
                           "d_demand_col": "Delta_Day_Ahead_Forecast_Demand",
                           "d_vre_col": "Delta_Day_Ahead_Forecast_VRE",
                          },
             "actual": {"demand_col": "Load",
                           "vre_col": "VRE",
                           "d_demand_col": "delta_Load",
                           "d_vre_col": "delta_VRE",
                          }
            }

for name, data_set_info in data_sets.items():
    demand_col = data_set_info["demand_col"]
    vre_col = data_set_info["vre_col"]
    d_demand_col = data_set_info["d_demand_col"]
    d_vre_col = data_set_info["d_vre_col"]
    # get rows that have a projection for all of: demand, vre, d_demand, d_vre
    mask = (~pd.isna(data_df.loc[:,demand_col])) & (~pd.isna(data_df.loc[:,vre_col])) \
           & (~pd.isna(data_df.loc[:,d_demand_col])) & (~pd.isna(data_df.loc[:,d_vre_col]))
    data_set_info["data_set_mask"] = mask
    
    # load this data to tensors for input features and labels
    feature_cols = [demand_col, vre_col, 'Hour', 'Day_of_Year', "Hours_since_2018"]
    label_col = 'delta_Total_CO2_Emissions'
    X = torch.tensor(data_df.loc[mask, feature_cols].values.astype(np.float32))
    y = torch.tensor(data_df.loc[mask, label_col].values.astype(np.float32))
    # scale input features
    X = torch.tensor(scaler.transform(X)).to(device)
    data_set_info["X"] = X
    data_set_info["y"] = y
    
    # also save the d_VRE and d_demand for each example hour
    data_set_info["d_demand"] = torch.tensor(data_df.loc[mask, d_demand_col].values.astype(np.float32))
    data_set_info["d_vre"] = torch.tensor(data_df.loc[mask, d_vre_col].values.astype(np.float32))

In [29]:
print("Number of example hours for which we have projected demand and VRE and both of their first derivs")
for name, data_info in data_sets.items():
    print(f"{name}: {len(data_info['y'])}")

Number of example hours for which we have projected demand and VRE and both of their first derivs
hour_ahead: 26268
day_ahead: 26006
actual: 26305


## Load saved model

In [30]:
def get_model(n_input, hidden_dims, n_out, dropout_p):
    
    layers = [nn.Linear(n_input, hidden_dims[0]),
              nn.BatchNorm1d(hidden_dims[0]),
              nn.ReLU(),
              nn.Dropout(dropout_p)
             ]
    for layer in range(len(hidden_dims)-1):
        cur_hidden, next_hidden = hidden_dims[layer], hidden_dims[layer+1]
        layers.extend([nn.Linear(cur_hidden, next_hidden),
                       nn.BatchNorm1d(next_hidden),
                       nn.ReLU(),
                       nn.Dropout(dropout_p)
                      ])
    layers.append(nn.Linear(hidden_dims[-1], n_out))
    
    model = nn.Sequential(*layers)
    return model


In [31]:
best_model_file = os.path.join(os.getcwd(), "ff_models", \
                               "hours_since_2018_feature", "2023_04_01-05_23_36_PM", \
                               "epoch=35779,r2=0.8933,Invalids=0.pth")


In [32]:
hidden_dims = [512,256]
n_out = 3 # intercept term
dropout_p = 0.5
n_input = len(feature_cols)

model = get_model(n_input, hidden_dims, n_out, dropout_p)
model.to(device)
model.load_state_dict(torch.load(best_model_file))
model.eval()

Sequential(
  (0): Linear(in_features=4, out_features=512, bias=True)
  (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=256, out_features=3, bias=True)
)

In [36]:
result_str = ""
for name, data_set_info in data_sets.items():
    X, y = data_set_info["X"], data_set_info["y"]
    # inference
    pred_coeff = model(X.float()).cpu()
    # split into mefs mdfs and intercept terms 
    mefs = pred_coeff[:, 0]
    mdfs = pred_coeff[:, 1]
    intercepts = pred_coeff[:, 2]
    data_set_info["pred_mef"] = mefs
    data_set_info["pred_mdf"] = mdfs
    data_set_info["pred_intercepts"] = intercepts
    # compute predicted change in emissions
    d_demand = data_set_info["d_demand"]
    d_vre = data_set_info["d_vre"]
    y_pred_demand = torch.mul(d_demand, mefs)
    y_pred_vre = torch.mul(d_vre, mdfs)
    y_pred = y_pred_vre + y_pred_demand + intercepts
    data_set_info["y_pred"] = y_pred
    # compute R2 and MAE
    r2 = r2_score(y.detach().numpy(), y_pred.detach().numpy())
    mae = mean_absolute_error(y.detach().numpy(), y_pred.detach().numpy())
    data_set_info["r2"] = r2
    data_set_info["mae"] = mae
    # count number of invalid coeffients predicted
    count_neg_mefs = torch.sum(mefs <= 0).item()  # MEF must be greater than 0
    count_pos_mdfs = torch.sum(mdfs > 0).item()  # MDF must be less than or equal to 0
    data_set_info["count_neg_mefs"] = count_neg_mefs
    data_set_info["count_pos_mdfs"] = count_pos_mdfs
    
    result_str += name
    result_str += f"\n\tR2: {r2:.4f}"
    result_str += f"\n\tMAE: {mae:.2f}"
    result_str += f"\n\tInvalid MEFs predicted: {count_neg_mefs}"
    result_str += f"\n\tInvalid MDFs predicted: {count_pos_mdfs}\n"
    
with open(os.path.join(work_dir, "results.txt"), 'w+') as f:
    f.write(result_str)

print(result_str)

hour_ahead
	R2: 0.3385
	MAE: 310913.22
	Invalid MEFs predicted: 0
	Invalid MDFs predicted: 0
day_ahead
	R2: 0.7284
	MAE: 206007.06
	Invalid MEFs predicted: 0
	Invalid MDFs predicted: 0
actual
	R2: 0.8907
	MAE: 131593.91
	Invalid MEFs predicted: 0
	Invalid MDFs predicted: 0



Hm looks kind of bad. Might want to inspect the projections to determine if this could be caused by some issues that could be filtered out. Then we could mention that sometime there are some trivially bad projections x% of the time. However idk, 33 r^2 seems like there are pervasive issues with the projection quality. It could also be an issue with the data pre-processing - maybe projections by CAISO weren't this bad, but we messed up somewhere in the process of creating the columns for average hourly projection and their deltas.

Putting the predictions into the table

In [41]:
for name, data_set_info in data_sets.items():
    mef_col = []
    mdf_col = []
    intercept_col = []
    pred_idx = 0
    for val in data_set_info["data_set_mask"]:
        if val:
            mef_col.append(data_set_info["pred_mef"][pred_idx].item())
            mdf_col.append(data_set_info["pred_mdf"][pred_idx].item())
            intercept_col.append(data_set_info["pred_intercepts"][pred_idx].item())
            pred_idx += 1
        else:
            mef_col.append(np.nan)
            mdf_col.append(np.nan)
            intercept_col.append(np.nan)
            
    data_df.loc[:, f"{name}_pred_mef"] = mef_col
    data_df.loc[:, f"{name}_pred_mdf"] = mdf_col
    data_df.loc[:, f"{name}_pred_intercept"] = intercept_col

In [42]:
data_df.to_csv(os.path.join(work_dir, "CAISO_Data_2019_2021_NN.with_historical_projections_and_coeff_preds.csv"))